Created on Sun Jan 12 00:23:08 2020
Group 7
@author: C.L.

In [1]:
import pandas as pd
import os
from matplotlib.pyplot import imread
import numpy as np
from PIL import Image

In [24]:
path = "./../CSV_annotate/SEATGURU/"
path_img = "./../Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/"
path_out = './../IMG_annot/SEATGURU/'

In [25]:
os.listdir(path_out)

['Others', 'Meal', 'Ext_Int', 'Ext', 'Int']

In [22]:
df_SEATGURU = pd.read_csv(path + 'g7_SEATGURU.csv', sep = ';', index_col = False)
df_SEATGURU.head()

,name,format,height,width,height_to_width,ncol,aircraft_manufacturer,aircraft_type
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,jpg,540,960,0.5625,3,Boeing,777
1,KLM_Airbus_A330-300_1.jpg,jpg,720,960,0.7500,3,Airbus,A330
2,American_Airlines_Boeing_767-300_3.jpg,jpg,720,720,1.0000,3,Boeing,767
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,jpg,720,960,0.7500,3,Boeing,767
4,United_Airlines_Q400_A_2.jpg,jpg,720,960,0.7500,3,Other,NaN


In [10]:
df_int = pd.read_csv(path + 'g7_SEATGURU_Int.csv', sep = ';')
df_int.rename(columns = {'View' : 'View_Int'}, inplace = True)
df_ext = pd.read_csv(path + 'g7_SEATGURU_Ext.csv', sep = ';')
df_ext.rename(columns = {'View' : 'View_Ext'}, inplace = True)
df_ext_int = pd.read_csv(path + 'g7_SEATGURU_Ext_Int.csv', sep = ';')
df_ext_int.rename(columns = {'View' : 'View_Ext_Int'}, inplace = True)
df_meal = pd.read_csv(path + 'g7_SEATGURU_Meal.csv', sep = ';')
df_meal.rename(columns = {'View' : 'View_Meal'}, inplace = True)
    

In [11]:
df_int.head()

,Picture name,View_Int
0,KLM_Airbus_A330-300_1.jpg,Int
1,American_Airlines_Boeing_767-300_3.jpg,Int
2,American_Airlines_Boeing_787-8_1.jpg,Int
3,Edelweiss_Air_A340-300_0.jpg,Int
4,Etihad_Airways_Airbus_A380_0.jpg,Int


In [12]:
print(len(df_int))
print(len(df_ext))
print(len(df_ext_int))
print(len(df_meal))

1873
275
175
174


In [13]:
df_merge = df_int.merge(df_ext, on = 'Picture name', how = 'outer')
df_merge = df_merge.merge(df_ext_int, on = 'Picture name', how = 'outer')
df_merge = df_merge.merge(df_meal, on = 'Picture name', how = 'outer')
df_merge.set_index('Picture name', inplace = True)
df_merge

,View_Int,View_Ext,View_Ext_Int,View_Meal
Picture name,,,,
KLM_Airbus_A330-300_1.jpg,Int,NaN,NaN,NaN
American_Airlines_Boeing_767-300_3.jpg,Int,NaN,NaN,NaN
American_Airlines_Boeing_787-8_1.jpg,Int,NaN,NaN,NaN
Edelweiss_Air_A340-300_0.jpg,Int,NaN,NaN,NaN
Etihad_Airways_Airbus_A380_0.jpg,Int,NaN,NaN,NaN
...,...,...,...,...
KLM_Boeing_747-400_combi_B_2.jpg,NaN,NaN,NaN,Meal
Iberia_Airbus_A319_3.jpg,NaN,NaN,NaN,Meal
Japan_Airlines_Boeing_767-300ER_V2_1.jpg,NaN,NaN,NaN,Meal


In [14]:
df_merge['full_count'] = df_merge.apply(lambda x : x.count(), axis = 1)
df_merge.head()

,View_Int,View_Ext,View_Ext_Int,View_Meal,full_count
Picture name,,,,,
KLM_Airbus_A330-300_1.jpg,Int,NaN,NaN,NaN,1
American_Airlines_Boeing_767-300_3.jpg,Int,NaN,NaN,NaN,1
American_Airlines_Boeing_787-8_1.jpg,Int,NaN,NaN,NaN,1
Edelweiss_Air_A340-300_0.jpg,Int,NaN,NaN,NaN,1
Etihad_Airways_Airbus_A380_0.jpg,Int,NaN,NaN,NaN,1


In [15]:
df_merge.columns

Index(['View_Int', 'View_Ext', 'View_Ext_Int', 'View_Meal', 'full_count'], dtype='object')

In [16]:
def crea_view(inte , ext , ext_int, meal, c):
    """
    Return the picture viewpoint. If it can't be labelled, return nan 
    
    Parameters :
        inte (type str) : 'Int' if interior viewpoint, else Nan 
        ext (type str) : 'Ext' if exterior viewpoint, else Nan 
        ext_int (type str) : 'Ext_Int' if the picture was taken from the inside but is depicting the outside, else Nan 
        meal (type str) : 'Meal' for a meal tray, else Nan 
        c : number of labels for this picture

    Out :
        v (type str) : picture viewpoint. If it can't be labelled, return nan           
    
    """
        
    v = np.nan
    if c == 1:
        l = [inte , ext , ext_int, meal]
        x = [el for el in l if str(el) != 'nan']
        v = x[0]
    return v
    
df_merge['view'] = df_merge.apply(lambda x : crea_view(x['View_Int'], x['View_Ext'], x['View_Ext_Int'], 
                                                       x['View_Meal'], x['full_count']), axis = 1)
df_merge

,View_Int,View_Ext,View_Ext_Int,View_Meal,full_count,view
Picture name,,,,,,
KLM_Airbus_A330-300_1.jpg,Int,NaN,NaN,NaN,1,Int
American_Airlines_Boeing_767-300_3.jpg,Int,NaN,NaN,NaN,1,Int
American_Airlines_Boeing_787-8_1.jpg,Int,NaN,NaN,NaN,1,Int
Edelweiss_Air_A340-300_0.jpg,Int,NaN,NaN,NaN,1,Int
Etihad_Airways_Airbus_A380_0.jpg,Int,NaN,NaN,NaN,1,Int
...,...,...,...,...,...,...
KLM_Boeing_747-400_combi_B_2.jpg,NaN,NaN,NaN,Meal,1,Meal
Iberia_Airbus_A319_3.jpg,NaN,NaN,NaN,Meal,1,Meal
Japan_Airlines_Boeing_767-300ER_V2_1.jpg,NaN,NaN,NaN,Meal,1,Meal


In [17]:
df_duplicate = df_merge[df_merge['full_count']>1]
len(df_duplicate)
for pic in df_duplicate.index.tolist():
    print(df_duplicate.loc[pic])
    img = imread(path_img + pic)
    plt.imshow(img)
    plt.show()

In [18]:
df_annot = df_SEATGURU.copy()
df_annot = df_annot.merge(df_merge['view'], left_on = 'name', right_index = True, how = 'left')
df_annot.head()

,name,format,height,width,height_to_width,ncol,aircraft_manufacturer,aircraft_type,view
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,jpg,540,960,0.5625,3,Boeing,777,Meal
1,KLM_Airbus_A330-300_1.jpg,jpg,720,960,0.7500,3,Airbus,A330,Int
2,American_Airlines_Boeing_767-300_3.jpg,jpg,720,720,1.0000,3,Boeing,767,Int
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,jpg,720,960,0.7500,3,Boeing,767,Meal
4,United_Airlines_Q400_A_2.jpg,jpg,720,960,0.7500,3,Other,NaN,Ext


In [395]:
# Generate folders with images depending on the labels

for el in [x for x in df_merge['view'].unique().tolist() if str(x) != 'nan']:
    os.makedirs(path_out + el +'/', exist_ok = True)
    df = df_annot[df_annot['view']==el]
    for pic in df['name'].tolist():
        try :
            img = Image.open(path_img + pic)
            img.save(path_out + el + '/' + pic)
        except :
            print(pic)
            pass

In [26]:
# Create the folder with the images without labels

os.makedirs(path_out + 'Others/', exist_ok = True)
df = df_annot[df_annot['view'].isna()]
print(len(df))
for pic in df['name'].tolist():
        try :
            img = Image.open(path_img + pic)
            img.save(path_out + 'Others/' + pic)
        except :
            print(pic)
            pass

59


In [ ]:
df_annot[]

In [27]:
df_annot.to_csv(path + 'g7_SEATGURU_annotate.csv', sep = ";", encoding='utf-8', index = False)